In [15]:
from utils import get_current_dir
import pandas as pd

inputpath = get_current_dir().parent / "data" / "artvis_cleaned.csv"
df = pd.read_csv(inputpath)

df = df.replace("null", pd.NA)
display(df.head())


df["a.id"] = df["a.id"].astype("Int64")
df = df.dropna(subset=["a.id"])

df["a.firstname"] = df["a.firstname"].astype("string")
df["a.firstname"] = df["a.firstname"].fillna("Unknown Firstname")

df["a.lastname"] = df["a.lastname"].astype("string")
df["a.lastname"] = df["a.lastname"].fillna("Unknown Lastname")

df["a.gender"] = pd.Categorical(df["a.gender"]).add_categories("Unknown Gender")
df["a.gender"] = df["a.gender"].fillna("Unknown Gender")

df["a.birthdate"] = pd.to_datetime(df["a.birthdate"], errors="coerce")

df["a.deathdate"] = pd.to_datetime(df["a.deathdate"], errors="coerce")

df["a.birthplace"] = df["a.birthplace"].astype("string")
df["a.birthplace"] = df["a.birthplace"].fillna("Unknown Birthplace")

df["a.deathplace"] = df["a.deathplace"].astype("string")
df["a.deathplace"] = df["a.deathplace"].fillna("Unknown Deathplace")

df["a.nationality"] = df["a.nationality"].astype("string")
df["a.nationality"] = df["a.nationality"].fillna("Unknown Nationality")

df["e.id"] = df["e.id"].astype("Int64")
df = df.dropna(subset=["e.id"])

df["e.title"] = df["e.title"].astype("string")
df["e.title"] = df["e.title"].fillna("Unknown Title")

df["e.venue"] = df["e.venue"].astype("string")
df["e.venue"] = df["e.venue"].fillna("Unknown Venue")

df["e.startdate"] = df["e.startdate"].astype("Int64")
df["e.startdate"] = df["e.startdate"].fillna(0)

df["e.type"] = pd.Categorical(df["e.type"]).add_categories("Unknown Type")
df["e.type"] = df["e.type"].fillna("Unknown Type")

df["e.paintings"] = df["e.paintings"].astype("Int64")
df["e.paintings"] = df["e.paintings"].fillna(0)

df["e.country"] = df["e.country"].astype("string")
df["e.country"] = df["e.country"].fillna("Unknown Country")

df["e.city"] = df["e.city"].astype("string")
df["e.city"] = df["e.city"].fillna("Unknown City")

df["e.latitude"] = df["e.latitude"].astype("float64")
df["e.latitude"] = df["e.latitude"].fillna(0)

df["e.longitude"] = df["e.longitude"].astype("float64")
df["e.longitude"] = df["e.longitude"].fillna(0)


print(df.head())
for col in df.columns:
    print(f"column: {col}, type: {df[col].dtype}, unique values: {df[col].nunique()}, null values: {df[col].isnull().sum()}")

,a.id,a.firstname,a.lastname,a.gender,a.birthdate,a.deathdate,a.birthplace,a.deathplace,a.nationality,e.id,e.title,e.venue,e.startdate,e.type,e.paintings,e.country,e.city,e.latitude,e.longitude
0,1,William Bernard,Adeney,M,1878-01-01,1966-01-01,London,London,GB,618,Exhibition of the Camden Town Group and Others,Public Art Galleries,1913,group,6,GB,Brighton,50.833333,-0.150000
1,1,William Bernard,Adeney,M,1878-01-01,1966-01-01,London,London,GB,720,The Second Exhibition of Works by Members of t...,Goupil Gallery,1915,group,4,GB,London,51.514248,-0.093145
2,1,William Bernard,Adeney,M,1878-01-01,1966-01-01,London,London,GB,729,Third Exhibition of Works by Members of the Lo...,Goupil Gallery,1915,group,5,GB,London,51.514248,-0.093145
3,1,William Bernard,Adeney,M,1878-01-01,1966-01-01,London,London,GB,650,The First Exhibition of Works by Members of Th...,Goupil Gallery,1914,group,5,GB,London,51.514248,-0.093145
4,1,William Bernard,Adeney,M,1878-01-01,1966-01-01,London,London,GB,680,Twentieth Century Art. A Review of Modern Move...,Whitechapel Art Gallery,1914,group,3,GB,London,51.514248,-0.093145


   a.id      a.firstname a.lastname a.gender a.birthdate a.deathdate  \
0     1  William Bernard     Adeney        M  1878-01-01  1966-01-01   
1     1  William Bernard     Adeney        M  1878-01-01  1966-01-01   
2     1  William Bernard     Adeney        M  1878-01-01  1966-01-01   
3     1  William Bernard     Adeney        M  1878-01-01  1966-01-01   
4     1  William Bernard     Adeney        M  1878-01-01  1966-01-01   

  a.birthplace a.deathplace a.nationality  e.id  \
0       London       London            GB   618   
1       London       London            GB   720   
2       London       London            GB   729   
3       London       London            GB   650   
4       London       London            GB   680   

                                             e.title                  e.venue  \
0     Exhibition of the Camden Town Group and Others     Public Art Galleries   
1  The Second Exhibition of Works by Members of t...           Goupil Gallery   
2  Third Exhibiti

In [16]:
outputpath = get_current_dir().parent / "pages"

# Vega Altair

In [28]:
import altair as alt
import pandas as pd

alt.data_transformers.enable("vegafusion")

location_stats = df.groupby(['e.country', 'e.city']).agg({
    'e.id': 'count',
    'e.paintings': 'sum',
    'e.venue': 'nunique',
    'e.latitude': 'first',
    'e.longitude': 'first'
}).reset_index()
location_stats.columns = ['country', 'city', 'num_exhibitions', 'total_paintings', 'unique_venues', 'latitude', 'longitude']

width = 1200  # Increased width
height = 600  # Increased height for the map

brush = alt.selection_interval()

# Base map layer using a world map dataset
base_map = alt.Chart(alt.topo_feature('https://cdn.jsdelivr.net/npm/world-atlas@2/countries-110m.json', 'countries')).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=width,
    height=height
).project('equirectangular')

# Exhibition locations layer
map_chart = alt.Chart(location_stats).mark_circle().encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    size=alt.Size('num_exhibitions:Q', 
                 scale=alt.Scale(range=[100, 2000]),  # Increased circle sizes
                 title='Number of Exhibitions'),
    color=alt.condition(brush, 
                       'country:N',
                       alt.value('lightgray')),
    tooltip=[
        alt.Tooltip('country:N', title='Country'),
        alt.Tooltip('city:N', title='City'),
        alt.Tooltip('num_exhibitions:Q', title='Number of Exhibitions'),
        alt.Tooltip('total_paintings:Q', title='Total Paintings'),
        alt.Tooltip('unique_venues:Q', title='Number of Venues')
    ]
).properties(
    width=width,
    height=height,
    title='Exhibition Locations'
).add_selection(brush)

# Combine base map and locations
combined_map = alt.layer(base_map, map_chart)

# Smaller charts below
small_height = 200  # Reduced height for bottom charts

country_chart = alt.Chart(location_stats).mark_bar().encode(
    x=alt.X('country:N', sort='-y', title='Country'),
    y=alt.Y('sum(num_exhibitions):Q', title='Number of Exhibitions'),
    color=alt.condition(brush, 'country:N', alt.value('lightgray')),
    tooltip=[
        alt.Tooltip('country:N', title='Country'),
        alt.Tooltip('sum(num_exhibitions):Q', title='Total Exhibitions'),
        alt.Tooltip('sum(total_paintings):Q', title='Total Paintings')
    ]
).properties(
    width=width//2,
    height=small_height
).transform_filter(brush)

scatter_chart = alt.Chart(location_stats).mark_circle(opacity=0.7).encode(
    x=alt.X('num_exhibitions:Q', title='Number of Exhibitions'),
    y=alt.Y('total_paintings:Q', title='Total Paintings'),
    color=alt.condition(brush, 'country:N', alt.value('lightgray')),
    size=alt.Size('unique_venues:Q', 
                 scale=alt.Scale(range=[50, 400]),
                 title='Number of Venues'),
    tooltip=[
        alt.Tooltip('country:N', title='Country'),
        alt.Tooltip('city:N', title='City'),
        alt.Tooltip('num_exhibitions:Q', title='Number of Exhibitions'),
        alt.Tooltip('total_paintings:Q', title='Total Paintings'),
        alt.Tooltip('unique_venues:Q', title='Number of Venues')
    ]
).properties(
    width=width//2,
    height=small_height
).transform_filter(brush)

final_visualization = alt.vconcat(
    combined_map,
    alt.hconcat(country_chart, scatter_chart)
).configure_view(
    stroke=None
).configure_title(
    fontSize=24,  # Increased font size
    anchor='middle'
).configure_legend(
    titleFontSize=12,
    labelFontSize=11
)

final_visualization.save(outputpath / "alt-final.html")

/var/folders/p6/sy85mkz13lsffnvz7237y4340000gn/T/ipykernel_52366/4130811501.py:50: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(brush)


# not so interesting

In [18]:
"""
works well
"""

import altair as alt
import pandas as pd

alt.data_transformers.enable("vegafusion")

location_stats = df.groupby(['e.country', 'e.city']).agg({
    'e.id': 'count',
    'e.paintings': 'sum',
    'e.venue': 'nunique',
    'e.latitude': 'first',
    'e.longitude': 'first'
}).reset_index()
location_stats.columns = ['country', 'city', 'num_exhibitions', 'total_paintings', 'unique_venues', 'latitude', 'longitude']

width = 800
height = 300

brush = alt.selection_interval()

map_chart = alt.Chart(location_stats).mark_circle().encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    size=alt.Size('num_exhibitions:Q', 
                 scale=alt.Scale(range=[50, 1000]),
                 title='Number of Exhibitions'),
    color=alt.condition(brush, 
                       'country:N',
                       alt.value('lightgray')),
    tooltip=[
        alt.Tooltip('country:N', title='Country'),
        alt.Tooltip('city:N', title='City'),
        alt.Tooltip('num_exhibitions:Q', title='Number of Exhibitions'),
        alt.Tooltip('total_paintings:Q', title='Total Paintings'),
        alt.Tooltip('unique_venues:Q', title='Number of Venues')
    ]
).properties(
    width=width,
    height=height,
    title='Exhibition Locations'
).add_selection(brush)

country_chart = alt.Chart(location_stats).mark_bar().encode(
    x=alt.X('country:N', sort='-y', title='Country'),
    y=alt.Y('sum(num_exhibitions):Q', title='Number of Exhibitions'),
    color=alt.condition(brush, 'country:N', alt.value('lightgray')),
    tooltip=[
        alt.Tooltip('country:N', title='Country'),
        alt.Tooltip('sum(num_exhibitions):Q', title='Total Exhibitions'),
        alt.Tooltip('sum(total_paintings):Q', title='Total Paintings')
    ]
).properties(
    width=width,
    height=height//2
).transform_filter(brush)

scatter_chart = alt.Chart(location_stats).mark_circle(opacity=0.7).encode(
    x=alt.X('num_exhibitions:Q', title='Number of Exhibitions'),
    y=alt.Y('total_paintings:Q', title='Total Paintings'),
    color=alt.condition(brush, 'country:N', alt.value('lightgray')),
    size=alt.Size('unique_venues:Q', 
                 scale=alt.Scale(range=[50, 400]),
                 title='Number of Venues'),
    tooltip=[
        alt.Tooltip('country:N', title='Country'),
        alt.Tooltip('city:N', title='City'),
        alt.Tooltip('num_exhibitions:Q', title='Number of Exhibitions'),
        alt.Tooltip('total_paintings:Q', title='Total Paintings'),
        alt.Tooltip('unique_venues:Q', title='Number of Venues')
    ]
).properties(
    width=width,
    height=height//2
).transform_filter(brush)

final_visualization = alt.vconcat(
    map_chart,
    alt.hconcat(country_chart, scatter_chart)
).configure_view(
    stroke=None
).configure_title(
    fontSize=20,
    anchor='middle'
)

# alt.renderers.enable('jupyter')
# display(alt.JupyterChart(final_visualization))

# final_visualization.save(outputpath / "alt-v1.html")

/var/folders/p6/sy85mkz13lsffnvz7237y4340000gn/T/ipykernel_52366/51330146.py:44: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(brush)


In [22]:
import altair as alt
import pandas as pd

alt.data_transformers.enable("vegafusion")

city_stats = df.groupby(['e.city', 'e.country']).agg({
    'e.id': 'count',
    'e.paintings': 'sum',
    'e.venue': 'nunique'
}).reset_index()
city_stats.columns = ['city', 'country', 'num_exhibitions', 'total_paintings', 'unique_venues']

width = 800
height = 300

brush = alt.selection_interval()

scatter = alt.Chart(city_stats).mark_circle(opacity=0.7).encode(
    x=alt.X('num_exhibitions:Q', title='Number of Exhibitions'),
    y=alt.Y('total_paintings:Q', title='Total Paintings'),
    size=alt.Size('unique_venues:Q', 
                 scale=alt.Scale(range=[100, 1000]),
                 title='Number of Venues'),
    color=alt.condition(brush,
                       alt.Color('country:N', scale=alt.Scale(scheme='category20')),
                       alt.value('lightgray')),
    tooltip=[
        alt.Tooltip('city:N', title='City'),
        alt.Tooltip('country:N', title='Country'),
        alt.Tooltip('num_exhibitions:Q', title='Number of Exhibitions'),
        alt.Tooltip('total_paintings:Q', title='Total Paintings'),
        alt.Tooltip('unique_venues:Q', title='Number of Venues')
    ]
).properties(
    width=width,
    height=height,
    title='Exhibition Distribution by City'
).add_selection(brush)

# Create the bar chart
bars = alt.Chart(city_stats).mark_bar().encode(
    x=alt.X('country:N', sort='-y', title='Country'),
    y=alt.Y('sum(num_exhibitions):Q', title='Total Exhibitions'),
    color=alt.condition(brush,
                       alt.Color('country:N', scale=alt.Scale(scheme='category20')),
                       alt.value('lightgray')),
    tooltip=[
        alt.Tooltip('country:N', title='Country'),
        alt.Tooltip('sum(num_exhibitions):Q', title='Total Exhibitions'),
        alt.Tooltip('sum(total_paintings):Q', title='Total Paintings')
    ]
).properties(
    width=width,
    height=height//1.5,
    title='Exhibitions by Country'
).transform_filter(
    brush
)

# Combine the charts
final_visualization = alt.vconcat(
    scatter,
    bars,
    title={
        "text": "Art Exhibition Analysis: Cities and Countries",
        "fontSize": 20,
        "anchor": "middle"
    }
).configure_view(
    stroke=None
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
)

# alt.renderers.enable('jupyter')
# display(alt.JupyterChart(final_visualization))

# final_visualization.save(outputpath / "alt-v5.html")

/var/folders/p6/sy85mkz13lsffnvz7237y4340000gn/T/ipykernel_52366/1464413008.py:43: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(brush)


# ignore

In [21]:
import altair as alt
import pandas as pd

alt.data_transformers.enable("vegafusion")

exhibition_stats = df.groupby(['e.city', 'e.country']).agg({
    'e.id': 'count',
    'e.paintings': 'sum',
    'a.id': 'nunique'
}).reset_index()
exhibition_stats.columns = ['city', 'country', 'num_exhibitions', 'total_paintings', 'unique_artists']

brush = alt.selection_interval()

width = 600
height = 300

scatter = alt.Chart(exhibition_stats).mark_circle(opacity=0.7).encode(
    x=alt.X('num_exhibitions:Q', title='Number of Exhibitions'),
    y=alt.Y('total_paintings:Q', title='Total Paintings'),
    size=alt.Size('unique_artists:Q', 
                 scale=alt.Scale(range=[100, 1000]),
                 title='Number of Unique Artists'),
    color=alt.condition(brush, 
                       'country:N',
                       alt.value('lightgray')),
    tooltip=[
        alt.Tooltip('city:N', title='City'),
        alt.Tooltip('country:N', title='Country'),
        alt.Tooltip('num_exhibitions:Q', title='Number of Exhibitions'),
        alt.Tooltip('total_paintings:Q', title='Total Paintings'),
        alt.Tooltip('unique_artists:Q', title='Number of Artists')
    ]
).properties(
    width=width,
    height=height,
    title='Exhibition Distribution'
).add_selection(brush)

bars = alt.Chart(exhibition_stats).mark_bar().encode(
    x=alt.X('city:N', sort='-y', title='City'),
    y=alt.Y('unique_artists:Q', title='Number of Artists'),
    color=alt.condition(brush, 
                       'country:N',
                       alt.value('lightgray')),
    tooltip=[
        alt.Tooltip('city:N', title='City'),
        alt.Tooltip('country:N', title='Country'),
        alt.Tooltip('unique_artists:Q', title='Number of Artists')
    ]
).transform_filter(
    brush
).properties(
    width=width,
    height=height,
    title='Artists by City (Filtered)'
)

final_visualization = alt.vconcat(
    scatter,
    bars
).configure_view(
    stroke=None
).configure_title(
    fontSize=16,
    anchor='middle'
)

# alt.renderers.enable('jupyter')
# display(alt.JupyterChart(final_visualization))

# final_visualization.save(outputpath / "alt-v4.html")

/var/folders/p6/sy85mkz13lsffnvz7237y4340000gn/T/ipykernel_52366/2087153479.py:38: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(brush)


In [26]:
import altair as alt
import pandas as pd

alt.data_transformers.enable("vegafusion")

artist_stats = df.groupby(['a.nationality', 'a.gender']).agg({
    'a.id': 'count',
    'e.paintings': 'sum'
}).reset_index()
artist_stats.columns = ['nationality', 'gender', 'num_artists', 'total_paintings']

width = 800
height = 400

selection = alt.selection_point(fields=['nationality'])

nationality_chart = alt.Chart(artist_stats).mark_bar().encode(
    x=alt.X('nationality:N', sort='-y', title='Nationality'),
    y=alt.Y('num_artists:Q', title='Number of Artists'),
    color=alt.condition(selection,
                       alt.Color('gender:N', scale=alt.Scale(scheme='category10')),
                       alt.value('gray')),
    tooltip=[
        alt.Tooltip('nationality:N', title='Nationality'),
        alt.Tooltip('gender:N', title='Gender'),
        alt.Tooltip('num_artists:Q', title='Number of Artists'),
        alt.Tooltip('total_paintings:Q', title='Total Paintings')
    ]
).properties(
    width=width,
    height=height//2
).add_selection(selection)

paintings_chart = alt.Chart(artist_stats).mark_circle(opacity=0.7).encode(
    x=alt.X('num_artists:Q', title='Number of Artists'),
    y=alt.Y('total_paintings:Q', title='Total Paintings'),
    size=alt.Size('num_artists:Q', scale=alt.Scale(range=[100, 1000])),
    color=alt.condition(selection,
                       alt.Color('gender:N', scale=alt.Scale(scheme='category10')),
                       alt.value('gray')),
    tooltip=[
        alt.Tooltip('nationality:N', title='Nationality'),
        alt.Tooltip('gender:N', title='Gender'),
        alt.Tooltip('num_artists:Q', title='Number of Artists'),
        alt.Tooltip('total_paintings:Q', title='Total Paintings')
    ]
).properties(
    width=width,
    height=height//2
).add_selection(selection)

final_visualization = alt.vconcat(
    nationality_chart,
    paintings_chart,
    title="Artist Distribution by Nationality and Gender"
).configure_view(
    stroke=None
).configure_title(
    fontSize=20,
    anchor='middle'
)

# alt.renderers.enable('jupyter')
# display(alt.JupyterChart(final_visualization))

# final_visualization.save(outputpath / "alt-v6.html")

/var/folders/p6/sy85mkz13lsffnvz7237y4340000gn/T/ipykernel_52366/2505813679.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  artist_stats = df.groupby(['a.nationality', 'a.gender']).agg({
/var/folders/p6/sy85mkz13lsffnvz7237y4340000gn/T/ipykernel_52366/2505813679.py:32: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(selection)
/var/folders/p6/sy85mkz13lsffnvz7237y4340000gn/T/ipykernel_52366/2505813679.py:50: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(selection)


In [ ]:
import altair as alt
import pandas as pd

alt.data_transformers.enable("vegafusion")

nationality_stats = df.groupby(['a.nationality']).agg({
    'a.id': 'count',
    'e.paintings': 'sum'
}).reset_index()
nationality_stats.columns = ['nationality', 'num_artists', 'total_paintings']

yearly_stats = df.groupby('e.startdate').agg({
    'e.id': 'count',
    'e.paintings': 'sum'
}).reset_index()
yearly_stats.columns = ['year', 'num_exhibitions', 'total_paintings']

selection = alt.selection_point(fields=['nationality'])

nationality_chart = alt.Chart(nationality_stats).mark_bar().encode(
    x=alt.X('nationality:N', sort='-y', title='Nationality'),
    y=alt.Y('num_artists:Q', title='Number of Artists'),
    color=alt.condition(selection,
                       alt.Color('total_paintings:Q', scale=alt.Scale(scheme='viridis')),
                       alt.value('gray')),
    tooltip=[
        alt.Tooltip('nationality:N', title='Nationality'),
        alt.Tooltip('num_artists:Q', title='Number of Artists'),
        alt.Tooltip('total_paintings:Q', title='Total Paintings')
    ]
).transform_window(
    rank='rank(num_artists)',
    sort=[alt.SortField('num_artists', order='descending')]
).transform_filter(
    alt.datum.rank <= 15
).properties(
    width=600,
    height=300,
    title='Top 15 Nationalities by Number of Artists'
).add_selection(selection)

time_chart = alt.Chart(yearly_stats).mark_line(point=True).encode(
    x=alt.X('year:Q', title='Year'),
    y=alt.Y('num_exhibitions:Q', title='Number of Exhibitions'),
    color=alt.value('steelblue'),
    tooltip=[
        alt.Tooltip('year:Q', title='Year'),
        alt.Tooltip('num_exhibitions:Q', title='Number of Exhibitions'),
        alt.Tooltip('total_paintings:Q', title='Total Paintings')
    ]
).properties(
    width=600,
    height=200,
    title='Exhibition Timeline'
)

final_vis = alt.vconcat(
    nationality_chart,
    time_chart
).configure_view(
    stroke=None
).configure_title(
    fontSize=16,
    anchor='middle'
)

# alt.renderers.enable('jupyter')
# display(alt.JupyterChart(final_vis))

# final_vis.save(outputpath / "alt-v7.html")

In [25]:
"""
pretty useless
"""

import altair as alt
import pandas as pd


alt.data_transformers.enable("vegafusion")

exhibitions_by_year = (
    df.groupby(['e.startdate', 'e.country', 'e.city'])
    .agg({
        'e.paintings': 'sum',
        'e.id': 'count'
    })
    .reset_index()
    .rename(columns={
        'e.id': 'num_exhibitions',
        'e.paintings': 'total_paintings'
    })
)

scatter = alt.Chart(exhibitions_by_year).mark_circle(size=60).encode(
    x=alt.X('num_exhibitions:Q', title='Number of Exhibitions'),
    y=alt.Y('total_paintings:Q', title='Total Paintings'),
    tooltip=['e.city:N', 'num_exhibitions:Q', 'total_paintings:Q']
).properties(
    width=1000,
    height=400,
    title='Exhibitions vs Paintings'
)

# scatter

# scatter.save(outputpath / "alt-v8.html")

In [ ]:
"""
kind of weird
"""

import altair as alt
import pandas as pd

alt.data_transformers.enable('vegafusion')

venue_stats = df.groupby(['e.venue', 'e.city']).agg({
    'e.id': 'count',
    'e.paintings': 'sum'
}).reset_index()
venue_stats.columns = ['venue', 'city', 'num_exhibitions', 'total_paintings']

time_stats = df.groupby('e.startdate').agg({
    'e.id': 'count',
    'e.paintings': 'sum'
}).reset_index()
time_stats.columns = ['year', 'num_exhibitions', 'total_paintings']

brush = alt.selection_interval(encodings=['x'])

width = 600
height = 200

timeline = alt.Chart(time_stats).mark_line(point=True).encode(
    x=alt.X('year:Q', title='Year'),
    y=alt.Y('num_exhibitions:Q', title='Number of Exhibitions'),
    tooltip=[
        alt.Tooltip('year:Q', title='Year'),
        alt.Tooltip('num_exhibitions:Q', title='Number of Exhibitions'),
        alt.Tooltip('total_paintings:Q', title='Total Paintings')
    ]
).properties(
    width=width,
    height=height
).add_selection(brush)

venue_chart = alt.Chart(venue_stats).mark_bar().encode(
    x=alt.X('venue:N', sort='-y', title='Venue', axis=alt.Axis(labels=False)),
    y=alt.Y('num_exhibitions:Q', title='Number of Exhibitions'),
    color=alt.condition(brush, 'city:N', alt.value('lightgray')),
    tooltip=[
        alt.Tooltip('venue:N', title='Venue'),
        alt.Tooltip('city:N', title='City'),
        alt.Tooltip('num_exhibitions:Q', title='Number of Exhibitions'),
        alt.Tooltip('total_paintings:Q', title='Total Paintings')
    ]
).properties(
    width=width,
    height=height
).transform_filter(brush)

scatter = alt.Chart(venue_stats).mark_circle(size=60).encode(
    x=alt.X('num_exhibitions:Q', title='Number of Exhibitions'),
    y=alt.Y('total_paintings:Q', title='Total Paintings'),
    color=alt.condition(brush, 'city:N', alt.value('lightgray')),
    tooltip=[
        alt.Tooltip('venue:N', title='Venue'),
        alt.Tooltip('city:N', title='City'),
        alt.Tooltip('num_exhibitions:Q', title='Number of Exhibitions'),
        alt.Tooltip('total_paintings:Q', title='Total Paintings')
    ]
).properties(
    width=width,
    height=height
).transform_filter(brush)

final_vis = alt.vconcat(
    timeline,
    venue_chart,
    scatter,
    title="Exhibition Analysis Over Time and Venues"
).configure_view(
    stroke=None
).configure_title(
    fontSize=20,
    anchor='middle'
)

# alt.renderers.enable('jupyter')
# final_vis

# final_vis.save(outputpath / "alt-v2.html")

In [ ]:
"""
kind of weird
"""

import altair as alt
import pandas as pd

alt.data_transformers.enable('vegafusion')

artist_exhibitions = df.groupby(['a.nationality', 'a.gender', 'e.startdate']).agg({
    'a.id': 'count',
    'e.paintings': 'sum'
}).reset_index()
artist_exhibitions.columns = ['nationality', 'gender', 'year', 'num_artists', 'total_paintings']

brush = alt.selection_interval(encodings=['x'])
click = alt.selection_point(fields=['nationality'])

width = 600
height = 200

timeline = alt.Chart(artist_exhibitions).mark_line(point=True).encode(
    x=alt.X('year:Q', title='Year'),
    y=alt.Y('total_paintings:Q', title='Total Paintings'),
    color=alt.Color('nationality:N', scale=alt.Scale(scheme='category20')),
    tooltip=['year:Q', 'nationality:N', 'total_paintings:Q']
).properties(
    width=width,
    height=height
).add_selection(brush)

nationality_chart = alt.Chart(artist_exhibitions).mark_bar().encode(
    x=alt.X('nationality:N', sort='-y', title='Nationality'),
    y=alt.Y('sum(num_artists):Q', title='Number of Artists'),
    color=alt.condition(click,
                       alt.Color('nationality:N', scale=alt.Scale(scheme='category20')),
                       alt.value('lightgray')),
    tooltip=['nationality:N', 'sum(num_artists):Q']
).properties(
    width=width,
    height=height
).add_selection(click).transform_filter(brush)

gender_scatter = alt.Chart(artist_exhibitions).mark_circle(size=60).encode(
    x=alt.X('sum(num_artists):Q', title='Number of Artists'),
    y=alt.Y('sum(total_paintings):Q', title='Total Paintings'),
    color=alt.condition(click,
                       alt.Color('gender:N', scale=alt.Scale(scheme='set2')),
                       alt.value('lightgray')),
    size=alt.Size('sum(num_artists):Q', scale=alt.Scale(range=[100, 1000])),
    tooltip=['gender:N', 'sum(num_artists):Q', 'sum(total_paintings):Q']
).properties(
    width=width,
    height=height
).transform_filter(brush)

final_vis = alt.vconcat(
    timeline,
    nationality_chart,
    gender_scatter,
    title={
        "text": "Art Exhibition Analysis (1913-1915)",
        "fontSize": 20,
        "anchor": "middle"
    }
).configure_view(
    stroke=None
)

# final_vis

# final_vis.save(outputpath / "alt-v2.html")